In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
from app.dependencies import create_dependencies, get_backtest_context_manager
create_dependencies()

context_mng = get_backtest_context_manager()

In [ ]:
context_mng.init_ready_contexts()

In [ ]:
import nest_asyncio
nest_asyncio.apply()
cnt = 0
for _, context in context_mng.contexts.items():
    await context.load_data_into_memory()
    cnt += 1
    if cnt == 10:
        break

In [ ]:
[x.ticker for x in context_mng.get_ready_contexts()]

In [ ]:
from app.backtest.evaluator import MultiContextFitnessEvaluator, SingleContextFitnessEvaluator
from app.backtest.action import MediumSwingActionMapper

In [ ]:
evaluator = SingleContextFitnessEvaluator(context_mng.get_context('NVDA'),
                                         MediumSwingActionMapper())

In [ ]:
from app.algorithms.structure.operators.construct import RandomMaxDepthInitOperator
from itapia_common.schemas.entities.rules import SemanticType
init_opr = RandomMaxDepthInitOperator(purpose=SemanticType.DECISION_SIGNAL, max_depth=6)

In [ ]:
from app.algorithms.structure.pop import Population
pop = Population(20)

In [ ]:
for _ in range(pop.population_size):
    pop.population.append(init_opr())

In [ ]:
for ind in pop.population:
    ind.cal_fitness(evaluator)

In [ ]:
for i, ind in enumerate(pop.population):
    print(f'Ind {id(ind)}: {ind.fitness}')